## Setup

Load libraries

In [1]:
import datetime
import pandas as pd
from snowai.density import (
    JonasDensity, 
    PistochiDensity, 
    SturmDensity,
    MachineLearningDensity
)

## Download Data

We will download some SNOTEL data using the `metloom API`. Install `metloom` using:

```bash
pip install metloom
```

In [4]:
try:
    from metloom.pointdata import SnotelPointData
except ImportError:
    !pip install metloom

In [7]:
# Let's look at our old friend banner summit
pt = SnotelPointData("312:ID:SNTL", "Banner Summit")

# start data and end date
start_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 2, 29)


# Notice this is a list
variables = [
    pt.ALLOWED_VARIABLES.SWE,
    pt.ALLOWED_VARIABLES.TEMPMIN,
    pt.ALLOWED_VARIABLES.TEMPMAX,
    pt.ALLOWED_VARIABLES.TEMPAVG,
    pt.ALLOWED_VARIABLES.SNOWDEPTH,
]

# request the data
df = pt.get_daily_data(start_date, end_date, variables)
df.head(3)

,,geometry,SWE,SWE_units,MIN AIR TEMP,MIN AIR TEMP_units,MAX AIR TEMP,MAX AIR TEMP_units,AVG AIR TEMP,AVG AIR TEMP_units,SNOWDEPTH,SNOWDEPTH_units,datasource
datetime,site,,,,,,,,,,,,
2024-01-01 08:00:00+00:00,312:ID:SNTL,POINT Z (-115.23447 44.30342 7040.00000),6.2,in,5.36,degF,33.08,degF,15.44,degF,25.0,in,NRCS
2024-01-02 08:00:00+00:00,312:ID:SNTL,POINT Z (-115.23447 44.30342 7040.00000),6.2,in,3.74,degF,28.58,degF,15.08,degF,25.0,in,NRCS
2024-01-03 08:00:00+00:00,312:ID:SNTL,POINT Z (-115.23447 44.30342 7040.00000),6.1,in,16.52,degF,28.40,degF,23.00,degF,25.0,in,NRCS


## Data Cleaning

We will do the following:

* Variable selection: we will select the variables of interest.
* Feature engineering:
    - we will convert inches to meters (all methods accepts snow depth in meters).
    - we will get snow class (Sturm and ML models need Snow Depth).

### Variable Selection